## Let's try out a neural network

## This turned into a weird experiment with using neural nets and regression to classify Fraudulant transactions

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
# from fastbook import *
# from utils import *
from fastai.tabular.all import *
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# #from dtreeviz.trees import *
# from IPython.display import Image, display_svg, SVG
# from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
# from sklearn.metrics import accuracy_score, roc_auc_score 
from sklearn.metrics import roc_auc_score 


path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 300

In [7]:
xs_final = (path/'xs_final.pkl').load()
train_ss = pd.read_csv(path/'train_ss.csv', index_col=[0], low_memory=False)
dep_var = 'isFraud'

In [8]:
df_nn = train_ss[list(xs_final.columns) + [dep_var]]

In [9]:
list(xs_final.columns)

['C5',
 'D3',
 'C14',
 'C13',
 'C4',
 'D2',
 'C1',
 'C8',
 'C2',
 'R_emaildomain',
 'D5',
 'M4',
 'card6',
 'C11',
 'C10',
 'TransactionDT',
 'id_20',
 'ProductCD',
 'M5',
 'C12',
 'TransactionAmt',
 'D1',
 'DeviceType',
 'D8',
 'addr2',
 'D15',
 'id_31',
 'C6',
 'card2']

In [6]:
assert len(xs_final.columns)==(len(df_nn.columns)-1)

In [7]:
df_nn.head()

,C5,D3,C14,C13,C4,D2,C1,C8,C2,R_emaildomain,D5,M4,card6,C11,C10,TransactionDT,id_20,ProductCD,M5,C12,TransactionAmt,D1,DeviceType,D8,addr2,D15,id_31,C6,card2,isFraud
57195,2.0,463.0,2.0,2.0,0.0,NaN,2.0,0.0,1.0,NaN,288.0,NaN,debit,1.0,0.0,1316341,NaN,W,NaN,0.0,418.95,472.0,NaN,NaN,87.0,288.0,NaN,1.0,490.0,0
312877,1.0,NaN,4.0,12.0,0.0,NaN,5.0,0.0,3.0,NaN,196.0,M0,debit,1.0,0.0,7780176,NaN,W,F,0.0,59.00,0.0,NaN,NaN,87.0,330.0,NaN,2.0,268.0,0
195195,0.0,0.0,1.0,8.0,0.0,51.0,1.0,0.0,2.0,NaN,0.0,M0,debit,2.0,0.0,4394717,NaN,W,T,0.0,29.00,51.0,NaN,NaN,87.0,51.0,NaN,1.0,417.0,1
21600,0.0,NaN,1.0,1.0,0.0,NaN,1.0,0.0,1.0,NaN,NaN,M1,debit,1.0,0.0,572518,NaN,W,T,0.0,24.00,0.0,NaN,NaN,87.0,0.0,NaN,0.0,194.0,0
12253,3.0,7.0,2.0,18.0,0.0,484.0,2.0,0.0,2.0,NaN,NaN,NaN,debit,3.0,0.0,348181,NaN,W,NaN,0.0,57.95,484.0,NaN,NaN,87.0,NaN,NaN,1.0,215.0,0


In [8]:
df_nn.columns

Index(['C5', 'D3', 'C14', 'C13', 'C4', 'D2', 'C1', 'C8', 'C2', 'R_emaildomain',
       'D5', 'M4', 'card6', 'C11', 'C10', 'TransactionDT', 'id_20',
       'ProductCD', 'M5', 'C12', 'TransactionAmt', 'D1', 'DeviceType', 'D8',
       'addr2', 'D15', 'id_31', 'C6', 'card2', 'isFraud'],
      dtype='object')

In [9]:
df_nn['isFraud']

57195     0
312877    0
195195    1
21600     0
12253     0
         ..
224165    0
322952    1
449491    0
343893    0
125898    0
Name: isFraud, Length: 59054, dtype: int64

In [10]:
cont = ['C5', 'D3', 'C14', 'C13', 'C4', 'D2', 'C1', 'C8', 'C2', 'D5', 'C11', 'C10',
        'C12', 'D1', 'D8', 'D15', 'C6', 'TransactionDT', 'TransactionAmt']
cat =  ['R_emaildomain', 'M4', 'M5', 'card6', 'id_20', 'ProductCD',
        'DeviceType', 'addr2', 'id_31', 'card2']

In [11]:
assert len(df_nn.columns)==(len(cat)+len(cont)+1)

In [12]:
df_nn[cat].nunique()

R_emaildomain     59
M4                 3
M5                 2
card6              4
id_20            206
ProductCD          5
DeviceType         2
addr2             27
id_31            107
card2            499
dtype: int64

There are NOT a numner of categorical variables where the number of categories has high cardinality, so deep learning might NOT help us in this senerio.

In [13]:
# The rows with TransactionDT <= 12579058 comprise the first 80% of the data (approximately)
cut_off = math.floor((train_ss['TransactionDT'].max() - train_ss['TransactionDT'].min())*0.8)
cut_off

12579784

In [14]:
cond = train_ss['TransactionDT'] <= cut_off
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx),list(valid_idx))

In [15]:
procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(df_nn, procs_nn, cat, cont,
                     splits=splits, y_names=dep_var)
to_nn.loss_func

FlattenedLoss of MSELoss()

In [16]:
to_nn['isFraud']

57195     0
312877    0
195195    1
21600     0
12253     0
         ..
585191    0
582517    0
493713    0
526363    0
500247    0
Name: isFraud, Length: 59054, dtype: int8

In [17]:
dls = to_nn.dataloaders(64)

In [18]:
dls.xs

,R_emaildomain,M4,M5,card6,id_20,ProductCD,DeviceType,addr2,id_31,card2,D3_na,D2_na,D5_na,D1_na,D8_na,D15_na,C5,D3,C14,C13,C4,D2,C1,C8,C2,D5,C11,C10,C12,D1,D8,D15,C6,TransactionDT,TransactionAmt
57195,0,0,0,3,0,5,0,21,0,390,1,2,1,1,2,1,-0.134384,9.471498,-0.123058,-0.224706,-0.061955,-0.280162,-0.089028,-0.056906,-0.090942,4.260183,-0.096037,-0.057582,-0.051114,2.468850,-0.154975,0.805232,-0.109859,-1.245707,1.240415
312877,0,1,1,3,0,5,0,21,0,168,2,2,1,1,2,1,-0.174292,-0.233869,-0.085467,-0.150241,-0.061955,-0.280162,-0.068799,-0.056906,-0.079297,2.774099,-0.096037,-0.057582,-0.051114,-0.590402,-0.154975,1.033601,-0.097012,0.519150,-0.328971
195195,0,1,2,3,0,5,0,21,0,317,1,1,1,1,2,1,-0.214200,-0.404513,-0.141853,-0.180027,-0.061955,-0.634464,-0.095771,-0.056906,-0.085119,-0.391905,-0.086453,-0.057582,-0.051114,-0.259847,-0.154975,-0.483425,-0.109859,-0.405201,-0.459771
21600,0,2,2,3,0,5,0,21,0,94,2,2,2,1,2,1,-0.214200,-0.233869,-0.141853,-0.232152,-0.061955,-0.280162,-0.095771,-0.056906,-0.090942,-0.246527,-0.096037,-0.057582,-0.051114,-0.590402,-0.154975,-0.760731,-0.122707,-1.448797,-0.481571
12253,0,0,0,3,0,5,0,21,0,115,1,1,2,1,2,2,-0.094476,-0.255200,-0.123058,-0.105562,-0.061955,2.700589,-0.089028,-0.056906,-0.085119,-0.246527,-0.076868,-0.057582,-0.051114,2.546627,-0.154975,-0.499737,-0.109859,-1.510049,-0.333549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224165,55,1,0,2,143,1,1,0,74,76,2,2,1,1,2,1,-0.214200,-0.233869,-0.123058,-0.224706,-0.048718,-0.280162,-0.089028,-0.038321,-0.085119,3.840203,-0.086453,-0.039016,-0.030764,-0.590402,-0.154975,-0.760731,-0.109859,-0.164251,-0.464052
322952,17,3,0,2,0,1,0,0,0,390,2,2,1,1,2,1,-0.214200,-0.233869,-0.160649,-0.239599,-0.048718,-0.280162,-0.095771,-0.047614,-0.085119,-0.391905,-0.086453,-0.048299,-0.030764,3.188292,-0.154975,-0.760731,-0.109859,0.582717,-0.047759
449491,0,1,2,3,0,5,0,21,0,320,1,1,1,1,2,1,-0.174292,-0.383183,-0.141853,-0.090669,-0.061955,3.039486,-0.095771,-0.056906,-0.090942,-0.375752,-0.096037,-0.057582,-0.051114,2.831812,-0.154975,2.115638,-0.109859,1.525638,-0.328971
343893,2,3,0,3,0,1,0,17,0,445,2,2,2,1,2,1,-0.214200,-0.233869,-0.141853,-0.232152,-0.048718,-0.280162,-0.095771,-0.056906,-0.090942,-0.246527,-0.096037,-0.057582,-0.040939,-0.590402,-0.154975,-0.760731,-0.109859,0.707703,-0.447118


In [19]:
learn = tabular_learner(dls, layers=[500,250], loss_func=F.mse_loss)

In [ ]:
learn.validate()

In [1]:
learn.loss_func

NameError: name 'learn' is not defined

In [2]:
learn.lr_find()

NameError: name 'learn' is not defined

In [3]:
learn.fit_one_cycle(5, 1e-3)

NameError: name 'learn' is not defined

In [417]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(y):
    preds = np.array(learn.get_preds()[0].squeeze())
    return r_mse(y, preds)

In [418]:
m_rmse(dls.valid_ds.y)

0.045772

To get the above result I treated the classification problem isFraud=1, ~isFraud=2 as a regression problem. Used MSELoss

In [421]:
preds = learn.get_preds()[0]
preds

tensor([[1.0257],
        [2.0897],
        [1.0185],
        ...,
        [1.0248],
        [1.0557],
        [0.9901]])

In [426]:
learn.get_preds()[0]>1.1

tensor([[False],
        [ True],
        [False],
        ...,
        [False],
        [False],
        [False]])

In [427]:
learn.get_preds()[1]>1.1

tensor([[False],
        [ True],
        [False],
        ...,
        [False],
        [False],
        [False]])

In [428]:
roc_auc_score(learn.get_preds()[1]>1.1, learn.get_preds()[0]>1.1)

0.9871521380781537

In [429]:
dls.xs.columns


Index(['R_emaildomain', 'M4', 'M5', 'card6', 'id_20', 'ProductCD',
       'DeviceType', 'addr2', 'id_31', 'card2', 'isFraud', 'D3_na', 'D2_na',
       'D5_na', 'D1_na', 'D8_na', 'D15_na', 'C5', 'D3', 'C14', 'C13', 'C4',
       'D2', 'C1', 'C8', 'C2', 'D5', 'C11', 'C10', 'C12', 'D1', 'D8', 'D15',
       'C6', 'TransactionDT', 'TransactionAmt'],
      dtype='object')